## WORKSHOP 6


### Conversione in PyPlasm del codice per la realizzazione del modello geometrico di Santo Stefano Rotondo al Celio

In [1]:
from pyplasm import *
from math import *

Evaluating fenvs.py..
...fenvs.py imported in 0.00633055708635 seconds


#### Misure di riferimento (cubito = 0.462 m)

In [2]:
r0 = 24
r2 = 48
r3 = 72
r4 = 88
wsteps = 10
nsteps = 10
wstep = wsteps/nsteps
hstep = 1/3
hw4 = 29
hbasament = hstep*nsteps

#### Mappa cilindrica

In [3]:
def cylMap():
    x= lambda k: k[1]*COS(k[0])
    y= lambda k: k[1]*SIN(k[0])
    z= lambda k: k[2]
    return MAP([x,y,z])

 


#### Scala esterna

In [4]:
def vdom(h):
    return PROD([COMP([EMBED(1),INTERVALS(PI*3/24)])(3),QUOTE([h])])

def hdom(w):
    return COMP([S(2)(-1), EMBED(1)])(PROD([INTERVALS(PI*3/24)(3), QUOTE([w])]))

def steps(w, h):
    def steps1(n):
        return COMP([STRUCT, CAT, N(n)])([vdom(h), T(3)(h), hdom(w), T(2)(-w)])
    return steps1

hbasement = hstep * nsteps

stair = RIGHT([COMP([MKPOL, UKPOL])(steps(wstep, hstep)(nsteps)),COMP([S(2)(-1), SKEL_2, CUBOID])([PI/24, wsteps, hbasament])])

ramp = cylMap()(T(2)(r4)(stair))
stairs = STRUCT(NN(12)([ramp, R([1, 2])(PI/6)]))

#### Basamento

In [5]:
basisSector = COMP([cylMap(),EMBED(1)])(PROD([INTERVALS(2*PI/12)(4),INTERVALS(r4-(wstep*(nsteps-1)))(1)]))

basis = COMP([STRUCT,NN(12)])([basisSector,R([1,2])(2*PI/12)])

basament = COMP([R([1,2])(PI/-48),STRUCT])([stairs,T(3)(hstep*nsteps),basis])

In [6]:
VIEW(basament)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000760EA50> >

#### Muro Esterno

In [7]:
extWall2Da = INTERSECTION([MKPOL([[[0,0],[7,0],[7,5],[0,8],[7,3],[9,4],[10.5,11.5],[10.5,0],[11,1.5],[11,3]],[[1,2,3,4],[2,8,7,6,5],[6,7,9,10]],[[1],[2],[3]]]),PROD([COMP([QUOTE,N(12)])(1),QUOTE([8])])])

extWall2Db = INTERSECTION([MKPOL([[[0.5,0],[4,0],[5,0],[6,0],[6,3],[5.5,3.5],[5,3],[4.5,3.5],[4,3],[2,4],[0,3],[0,1.5],[0.5,1.5]],[[10,11,12,13],[1,2,9,10,13],[2,3,7,8,9],[3,4,5,6,7]],[1,2,3,4]]),PROD([COMP([QUOTE,N(6)])(1),QUOTE([4])])])

extWall2D = STRUCT([extWall2Da,T(1)(11),extWall2Db])

sizxExtWall = SIZE(1)(extWall2D)

extWall = R([2,3])(PI/2)(PROD([extWall2D,QUOTE([1.5])]))

curvedExtWall = COMP([cylMap(),T(2)(r3),S([1,3])([PI/(4*sizxExtWall),hw4/8])])(extWall)

doubleExtWall = STRUCT([curvedExtWall,S(1)(-1),curvedExtWall])

fullExtWall = COMP([STRUCT,NN(4)])([doubleExtWall,R([1,2])(PI/2)])



In [8]:
VIEW(STRUCT([basament,T(3)(hbasement),fullExtWall]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000CFBF060> >

#### Colonne Intermedie

In [104]:
def column(args):
    w,h = args
    basis = CUBOID([w,w,2*w/3])
    trunk = CYLINDER([w/2*0.85,h-w])(8)
    capitel = CUBOID([w,w,w/3])
    return TOP([basis,TOP([trunk,capitel])])

arcAngle = 2*PI/50.4

wallAngle = -3.2*arcAngle/4

rotCross = 3.2*arcAngle/4 + 2.5*arcAngle

hCol = 12

myColumn = COMP([MKPOL,UKPOL,T(2)((r2-0.75)),column])([1.5,hCol])

the4Cols = COMP([R([1,2])(arcAngle*0.4/3.2),MKPOL,UKPOL,STRUCT,NN(4)])([R([1,2])(-arcAngle),myColumn])

the5Cols = COMP([R([1,2])(arcAngle*0.4/3.2),MKPOL,UKPOL,STRUCT,NN(5)])([R([1,2])(-arcAngle),myColumn])

theBotWal = COMP([cylMap(),MKPOL,UKPOL,T(2)(r2-0.75),CUBOID])([3.2*arcAngle/4,1.5,hCol])

theSecCols = STRUCT([R([1,2])(rotCross),theBotWal,R([1,2])(wallAngle),the4Cols,R([1,2])(-5*arcAngle),theBotWal,R([1,2])(wallAngle),the5Cols])

theMedColumns = COMP([STRUCT,NN(4)])([theSecCols,R([1,2])(PI/-2)])

In [132]:
VIEW(STRUCT([basament,T(3)(hbasement),fullExtWall,theMedColumns]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D3FCA50> >

#### Archi sulle colonne

In [130]:
def bottomArc(d):
    return BEZIER(S1)([[0,0],[0,2*d/3],[d,2*d/3],[d,0]])

def topArc(d):
    return BEZIER(S1)([[0,2*d/3],[d,2*d/3]])

def arc2D(d):
    return BEZIER(S2)([bottomArc(d),topArc(d)])

def arc3D(d):
    def arc3D1(w):
        return COMP([T(2)(w),R([2,3])(PI/2)])(PROD([QUOTE([w]),MAP(arc2D(d))(PROD([INTERVALS(1)(8),INTERVALS(1)(1)]))]))
    return arc3D1

def interarc(d1,d2):
    def interarc1(w):
        return CUBOID([d1,w,2*d2/3])
    return interarc1

def xArc(args):
    d1,d2 = args
    def xArc1(w):
        return RIGHT([interarc(d1,d2)(w),RIGHT([arc3D(d2)(w),interarc(d1,d2)(w)])])
    return xArc1

theArc = xArc([0.4,3.2])(1.5)

sizeArc = SIZE(1)(theArc)

def curvedArc():
    sx = (1/sizeArc)*arcAngle
    sz = sx*(r2-2)
    return COMP([cylMap(),MKPOL,UKPOL,T(2)(r2-0.75),S([1,3])([sx,sz])])(theArc)

heightArc = SIZE(3)(curvedArc())

the5arcs = COMP([MKPOL,UKPOL,STRUCT,NN(5)])([curvedArc(),R([1,2])(-arcAngle)])

the6arcs = COMP([MKPOL,UKPOL,STRUCT,NN(6)])([curvedArc(),R([1,2])(-arcAngle)])

theMidWall = COMP([cylMap(),MKPOL,UKPOL,T(2)(r2-0.75),CUBOID])([3.2*arcAngle/4,1.5,heightArc])

theSector = STRUCT([R([1,2])(rotCross),theMidWall,R([1,2])(wallAngle),the5arcs,R([1,2])(-5*arcAngle),theMidWall,R([1,2])(wallAngle),the6arcs])

theArcs = COMP([STRUCT,NN(4)])([theSector,T([1,2])(PI/-2)])

theArcsCols = STRUCT([theMedColumns,T(3)(hCol),theArcs])


In [131]:
VIEW(STRUCT([basament,T(3)(hbasement),fullExtWall,theArcsCols]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D0E6AB0> >